In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from talib import RSI, BBANDS, MACD

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
sns.set_style('whitegrid')
idx = pd.IndexSlice

## Data Retrieval

In [3]:
DATA_STORE = '../../data/assets.h5'

In [4]:
with pd.HDFStore(DATA_STORE) as store:
    data = (store['alpaca/tesla/prices']
            .loc[idx['2018':'2023', 'TSLA'],
                 ['open', 'high', 'low', 'close', 'volume']]
            .unstack('symbol')
            .swaplevel(axis=1)
            .loc[:, 'TSLA'])

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 805588 entries, 2018-01-02 09:48:00+00:00 to 2022-10-06 23:59:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   open    805588 non-null  float64
 1   high    805588 non-null  float64
 2   low     805588 non-null  float64
 3   close   805588 non-null  float64
 4   volume  805588 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 36.9+ MB


In [6]:
display(data.head())
display(data.tail())

,open,high,low,close,volume
timestamp,,,,,
2018-01-02 09:48:00+00:00,312.00,312.00,312.00,312.00,200
2018-01-02 09:50:00+00:00,311.80,311.80,311.80,311.80,300
2018-01-02 10:28:00+00:00,311.50,311.50,311.50,311.50,847
2018-01-02 11:07:00+00:00,310.99,310.99,310.99,310.99,307
2018-01-02 11:09:00+00:00,310.99,310.99,310.99,310.99,126


,open,high,low,close,volume
timestamp,,,,,
2022-10-06 23:55:00+00:00,236.83,236.84,236.75,236.75,5301
2022-10-06 23:56:00+00:00,236.71,236.71,236.52,236.62,13868
2022-10-06 23:57:00+00:00,236.62,236.62,236.53,236.59,8869
2022-10-06 23:58:00+00:00,236.60,236.78,236.60,236.63,6813
2022-10-06 23:59:00+00:00,236.72,236.80,236.65,236.70,9255


## Compute Bollinger Bands

In [7]:
# Compute Bollinger Bands
up, mid, low = BBANDS(data.close, timeperiod=21, nbdevup=2, nbdevdn=2, matype=0)

# Visualization 
data[['close', 'open']].plot(figsize=(16, 9), title='Bollinger Bands')
plt.plot(up, label='Upper Band')
plt.plot(mid, label='Middle Band')
plt.plot(low, label='Lower Band')
plt.legend(loc='best')
plt.show()

In [ ]:
# Save plot of Bollinger Bands as jpeg in the images folder
plt.savefig('../images/bollinger_bands_tesla.jpeg')

## Compute Relative Strength Index

In [ ]:
rsi = RSI(data.close, timeperiod=14)

In [ ]:
# Compute RSI index
rsi = RSI(data.close, timeperiod=14)

# Plot RSI
fig, ax = plt.subplots(figsize=(21, 15))
ax.plot(data.index, rsi, label='RSI')
ax.axhline(70, color='red')
ax.axhline(30, color='green')
ax.set_ylabel('RSI')
ax.legend(loc='best')
ax.grid(True)
plt.show()

In [ ]:
# Save RSI plot as jpeg
plt.savefig('../images/rsi_tesla.jpeg')

## Moving Average Convergence/Divergence

In [ ]:
macd, macdsignal, macdhist = MACD(data.close, fastperiod=12, slowperiod=26, signalperiod=9)

In [ ]:
macd_data = pd.DataFrame({'TSLA': data.close, 'TSLA': macd, 'MACD Signal': macdsignal, 'MACD History': macdhist})

fig, axes= plt.subplots(nrows=2, figsize=(21, 15))
macd_data.TSLA.plot(ax=axes[0])
macd_data.drop('TSLA', axis=1).plot(ax=axes[1])
fig.tight_layout()
sns.despine();

In [ ]:
# Save MCA plot as jpeg
plt.savefig('../images/macd_tesla.jpeg')

## Plot Result

In [ ]:
data = pd.DataFrame({'TSLA': data.close, 'BB Up': up, 'BB Mid': mid, 'BB down': low, 'RSI': rsi, 'MACD': macd})

In [ ]:
fig, axes= plt.subplots(nrows=3, figsize=(21, 15), sharex=True)
data.drop(['RSI', 'MACD'], axis=1).plot(ax=axes[0], lw=1, title='Bollinger Bands')
data['RSI'].plot(ax=axes[1], lw=1, title='Relative Strength Index')
axes[1].axhline(70, lw=1, ls='--', c='k')
axes[1].axhline(30, lw=1, ls='--', c='k')
data.MACD.plot(ax=axes[2], lw=1, title='Moving Average Convergence/Divergence', rot=0)
axes[2].set_xlabel('')
fig.tight_layout()
sns.despine();

In [ ]:
# Save combined plot as jpeg
plt.savefig('../images/combined_tesla.jpeg')